In [33]:
import pandas as pd
from string import ascii_uppercase as alfabeto

Importamos el archivo desde Wikipedia.


In [9]:
tablas=pd.read_html('https://web.archive.org/web/20221115040351/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup')

Los grupos van de la A a la H
Entonces: 11->7*8+11=67

In [19]:
todas_las_tablas[11]
todas_las_tablas[18].columns

Index(['Netherlands', 'Match 36', 'Qatar'], dtype='object')

In [25]:
tablas=pd.read_html('https://web.archive.org/web/20221115040351/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup')

dict_tablas = {}
for letra,i in zip(alfabeto,range[13, 67, 7]):
    df = tablas [i]
    df.rename(columns={df.columns[1]: 'Team'}, inplace=True)
    df.pop('qualification')
    dict_tablas[f'Grupo{letra}']
print(df)


KeyboardInterrupt: 

Extraemos la data de la pagina Web

In [27]:
from bs4 import BeautifulSoup
import requests 

In [35]:
years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974, 1978, 1982,
          1986, 1990, 1994, 1998, 2002,
         2006, 2010, 2014, 2018, 2022 ]

In [37]:
def get_matches(year):

    web = f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'
    response = requests.get(web)
    content = response.text

    soup = BeautifulSoup(content, 'lxml')

    matches = soup.find_all('div', class_='footballbox')


    home = []
    score = []
    away = []


    for match in matches:
        home.append(match.find('th', class_='fhome').get_text())
        score.append(match.find('th', class_='fscore').get_text())
        away.append(match.find('th', class_='faway').get_text())

    dict_football = {'home':home, 'score':score, 'away':away}
    df_football = pd.DataFrame(dict_football)
    df_football['year'] = year
    return df_football


#Historical Data
fifa = [get_matches(year) for year in years]
df_fifa = pd.concat(fifa, ignore_index=True)
df_fifa.to_csv('fifa_worlcup_historical_data.csv', index=False)

#fixture
df_fixture = get_matches('2022')
df_fixture.to_csv('fifa_worlcup_fixture.csv', index=False)




  

Segunda parte de web scraping

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import service
import pandas as pd
import time 

Extraccion de datos de distintos años con weebscraping

In [3]:
years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974, 1978, 1982,
          1986, 1990, 1994, 1998, 2002,
         2006, 2010, 2014, 2018, 2022 ]

path = "C:\Users\Notebook\Desktop\Crome\chromedriver.exe"
service = service(executable_path=path)
driver = webdriver.chrome(service=service)

def obtener_data_faltante(year):   
    web = f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup#Qualification'



    driver.get(web)

    matches = driver.find_elements(by= 'xpath', value='//td[@align='right']/.. | //td[@style="text-aling:right;"]/..')

    home = []
    score = []
    away = []


    for match in matches:
        home.append(match.find_element(by='xpath', value='./td[1]').text)
        score.append(match.find_element(by='xpath', value='./td[2]').text)
        away.append(match.find_element(by='xpath', value='./td[3]').text)

    dict_football = {'home': home, 'score': score, 'away':away}
    df_football = pd.DataFrame(dict_football)
    df_football['year'] = year
    time.sleep(2)
    return df_football
    

years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974, 1978, 1982,
          1986, 1990, 1994, 1998, 2002,
         2006, 2010, 2014, 2018, 2022 ]

fifa = [obtener_data_faltante(year) for year in years]
driver.quit()

df_fifa = pd.concat(fifa,ignore_index=True)
df_fifa.to_csv('fifa_woldcup_missing_data.csv', index=False)




SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (1266635707.py, line 1)

Limpieza de los datos

In [1]:
import pandas as pd

In [3]:
df_data_historica = pd.read_csv('fifa_worldcup_matches.csv')
df_fixture = pd.read_csv('fifa_worldcup_fixture.csv')
df_data_faltante = pd.read_csv('fifa_worldcup_missing_data.csv')

1.1: Limpiando df_fixture

In [8]:
df_fixture
#Problemas en home y away

#Eliminamos espacios en blanco
df_fixture['home'] = df_fixture['home'].str.strip()
df_fixture['away'] = df_fixture['away'].str.strip() 



1.2:Limpiando df_missing_value y agregandolo a df_historical_data

In [12]:
#df_data_faltante[df_data_faltante['home'].isnull()] Encontramos data faltante, procedemos a eliminarla
df_data_faltante.dropna(inplace=True)

In [17]:
#Concatenando los dos Dataframes
df_data_historica = pd.concat([df_data_faltante, df_data_historica], ignore_index=True)
#Eliminamos los datos duplicados
df_data_historica.drop_duplicates(inplace=True)
#Ordenamos por año
df_data_historica.sort_values('year', inplace=True)
df_data_historica


,home,score,away,year
450,Uruguay,4–0,Romania,1930
460,Argentina,1–0,France,1930
459,Romania,3–1,Peru,1930
458,Chile,3–0,Mexico,1930
457,Chile,1–0,France,1930
...,...,...,...,...
1307,Russia,2–2 (a.e.t.),Croatia,2018
1308,France,1–0,Belgium,2018
1309,Croatia,2–1 (a.e.t.),England,2018
1311,Sweden,1–0,South Korea,2018


1.3: Limpiando df_historical_data

In [19]:
index_eliminar = df_data_historica[df_data_historica['home'].str.contains('sweden') &
                                   df_data_historica['away'].str.contains('austria')].index


,home,score,away,year


In [20]:
df_data_historica.drop(index=index_eliminar, inplace=True)


NameError: name 'index_eliminar' is not defined

In [22]:
df_data_historica = df_data_historica['score'].str.contains('[^\d]') #('[^\d]'): Encuentra caracteres que no sean digitos o -


In [23]:
df_data_historica = df_data_historica['score'].str.replace('[^\d]','', regex=True)


KeyboardInterrupt



In [27]:
df_data_historica

450     True
460     True
459     True
458     True
457     True
        ... 
1307    True
1308    True
1309    True
1311    True
1344    True
Name: score, Length: 901, dtype: bool

In [25]:
df_fixture['home'] = df_fixture['home'].str.strip()
df_fixture['away'] = df_fixture['away'].str.strip()

In [28]:
df_data_historica[['HomeGoals', 'AwayGoals']]= df_data_historica['score'].str.split('-', expand=True)

KeyError: 'score'

In [ ]:
df_data_historica.drop('score', axis=1, inplace = True)

In [ ]:
df_data_historica.rename(columns= {'home':'HomeTeam','away':'AwayTeam','year':'Year'}, inplace=True)


In [ ]:
df_data_historica.dtypes
df_data_historica= df_data_historica.astype({'HomeGoals': int, 'AwayGoals':int})

In [ ]:
df_data_historica.to_csv('clean_fifa_worldcup_matches.csv', index=False)
df_fixture.to_csv('clean_fifa_worldcup_fixture.csv', index=False)